In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import mie_params_of_vle_visc as mie_params_of


PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
#######################################
# type and optimization configuration #
#######################################

of_type = "vle_visc"
# ranges for sigma, eps, lambda_r
l_bounds = [2.5, 100., 9.]
u_bounds = [3.5, 300., 26.]
m_base = 5
params_file = f'optimized_mie_params_{of_type}.csv'

In [3]:
filename = '../../computed_files/phase_equilibria_fluid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp}

In [4]:
######################
# Loading FE-ANN EoS #
######################

ckpt_folder = '../../ann_models/feann_eos'

prefix_params = 'FE-ANN-EoS-params_'

###
Tscale = 'Tinv'
seed = 1
factor = 0.05
EPOCHS = 20000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}
fun_dic_fluid = helper_jitted_funs(helmholtz_model, helmholtz_params)

#########################
# Shear visocsity model #
#########################

activation_dicts = {'linear': linear_activation, 'softplus': nn.softplus}
folder_visc = '../../ann_models/visc_models'
hidden_layers = 2
neurons = 30
prefix = 'logvisc-rho-Tinv-penalty'
ckpt_dir = folder_visc
seed = 0
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logvisc = {'params': state_restored['params']}
logvisc_model = TransportModel_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

logvisc_model_jit = jax.jit(lambda alpha, rhoad, Tad: logvisc_model.apply(params_logvisc, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
visc_fun = lambda alpha, rhoad, Tad: jnp.exp(logvisc_model_jit(alpha, rhoad, Tad))

In [5]:
# Reading data from NIST
filename = 'krypton.xlsx'
DataFile = pd.ExcelFile(filename)

if not os.path.exists(params_file):
    sampler = qmc.Sobol(d=3, scramble=False)
    sample = sampler.random_base2(m=m_base)

    scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

    sigma_guess = scaled_sample[:, 0]
    epsilon_guess = scaled_sample[:, 1]
    lr_guess = scaled_sample[:, 2]

    n = len(sigma_guess)

    index = np.arange(n)
    solved = np.zeros(n, dtype=bool)
    sigma_sol = np.nan * np.ones(n)
    epsilon_sol = np.nan * np.ones(n)
    lr_sol = np.nan * np.ones(n)
    of = np.nan * np.ones(n)
    solved_success = np.zeros(n, dtype=bool)

    df = pd.DataFrame({'index': index, 
                    'sigma_guess': sigma_guess, 'epsilon_guess': epsilon_guess, 'lr_guess': lr_guess,
                    'optimized': solved, 
                    'sigma_sol': sigma_sol, 'epsilon_sol': epsilon_sol, 'lr_sol': lr_sol, 'of': of, 'solved_success': solved_success})
    
    df.to_csv(params_file, index=False)




In [6]:
# bounds to control sigma, epsilon and lr
bounds = ((None, None), (None, None), (7., 34.))

kwargs = {'DataFile': DataFile,
          'fun_dic': fun_dic_fluid,
          'visc_fun': visc_fun,
          'interpd_dict': interpd_dict,
          'lambda_a': 6.,
          'weight_rhov_vle':0.0,
          'weight_hvap':1.0,
          'add_critical_point': True,
          'weight_visc': 1.0,   
          }
args = tuple(kwargs.values())

In [7]:
df = pd.read_csv(params_file)
n = len(df)
for i in range(n):
    if not df.loc[i, 'optimized']:
        inc0 = df.loc[i, ['sigma_guess', 'epsilon_guess', 'lr_guess']].to_list()
        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds, options={'maxiter':10})

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)
    print(i, df.loc[i, 'optimized'], df.loc[i, 'of'])

0 True 0.4222061931585392
1 True 0.0726113413628767
2 True 0.0038049754157412
3 True 0.9233302632549468
4 True 0.0654900238071575
5 True 0.3581113297098827
6 True 3.4733675029173123
7 True 1.2330648844344088
8 True 0.2798090498632388
9 True 0.0601873047846248
10 True 3.4729926353991525
11 True 0.0946811297850143
12 True 0.2841659972885698
13 True 0.1209131402089369
14 True 0.0424143227409599
15 True 2.8138744941138634
16 True 0.0690648121762801
17 True 0.2918158582379027
18 True 0.0038049754150875
19 True 0.1538091578560577
20 True 3.483918002902984
21 True 0.0038049754150741
22 True 0.0038049754151058
23 True 0.5832724596002432
24 True 0.7063896328827274
25 True 0.0038049754150489
26 True 0.0612813649829785
27 True 0.2233936013828328
28 True 0.0876118800414748
29 True 0.0863054561350033
30 True 3.788188810718424
31 True 0.1618230079748066


In [8]:
df = pd.read_csv(params_file)
n = len(df)

of_order = np.argsort(df['of'].to_numpy())

for i in of_order[:5]:
    if not df.loc[i, 'solved_success']:
        inc0 = df.loc[i, ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()

        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds)

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)


In [9]:
of_order = np.argsort(df['of'].to_numpy())
df.loc[of_order].head(n=10)

,index,sigma_guess,epsilon_guess,lr_guess,optimized,sigma_sol,epsilon_sol,lr_sol,of,solved_success
25,25,3.15625,231.25,9.53125,True,3.676797,169.415903,12.892855,0.003805,True
21,21,3.46875,218.75,14.84375,True,3.676797,169.415880,12.892852,0.003805,True
18,18,3.34375,143.75,12.71875,True,3.676797,169.415869,12.892850,0.003805,True
22,22,3.21875,168.75,19.09375,True,3.676797,169.415898,12.892854,0.003805,True
2,2,3.25000,150.00,13.25000,True,3.676797,169.416014,12.892872,0.003805,True
14,14,3.06250,187.50,10.06250,True,3.521665,149.800669,10.394121,0.042414,False
9,9,3.18750,262.50,16.43750,True,3.873097,188.269033,17.253738,0.060187,False
26,26,3.40625,181.25,22.28125,True,3.756687,200.245199,19.049857,0.061281,False
4,4,2.87500,175.00,19.62500,True,3.593750,183.750000,15.700000,0.065490,False
16,16,2.59375,193.75,16.96875,True,3.525579,174.375000,13.103646,0.069065,False


In [10]:
param_best = df.loc[of_order[0], ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()
param_best

[3.676797337742965, 169.41590280240456, 12.892855204296222]